In [1]:
!pip install qiskit_aer
!pip install qiskit_ibm_runtime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qiskit import *
from qiskit.visualization import *
from qiskit_ibm_runtime import *

service = QiskitRuntimeService()



In [2]:
# Select the backend 
backend_name = 'ibm_brisbane'
backend = service.backend(backend_name)

# Get the backend properties
properties = backend.properties()

# Display the properties
print(properties)

{'backend_name': 'ibm_brisbane', 'backend_version': '1.1.40', 'last_update_date': datetime.datetime(2024, 10, 11, 18, 30, 9, tzinfo=tzlocal()), 'qubits': [[{'date': datetime.datetime(2024, 10, 11, 4, 6, 3, tzinfo=tzlocal()), 'name': 'T1', 'unit': 'us', 'value': 258.3935194932953}, {'date': datetime.datetime(2024, 10, 11, 4, 9, 15, tzinfo=tzlocal()), 'name': 'T2', 'unit': 'us', 'value': 59.46668776727226}, {'date': datetime.datetime(2024, 10, 11, 18, 30, 9, tzinfo=tzlocal()), 'name': 'frequency', 'unit': 'GHz', 'value': 4.721908316964448}, {'date': datetime.datetime(2024, 10, 11, 18, 30, 9, tzinfo=tzlocal()), 'name': 'anharmonicity', 'unit': 'GHz', 'value': -0.31197865973435573}, {'date': datetime.datetime(2024, 10, 11, 4, 0, 35, tzinfo=tzlocal()), 'name': 'readout_error', 'unit': '', 'value': 0.023600000000000065}, {'date': datetime.datetime(2024, 10, 11, 4, 0, 35, tzinfo=tzlocal()), 'name': 'prob_meas0_prep1', 'unit': '', 'value': 0.0218}, {'date': datetime.datetime(2024, 10, 11, 4, 0

NameError: name 'datetime' is not defined

In [6]:
df = pd.read_csv("ibm_brisbane_calibrations_2024-10-11T13_37_10Z.csv")

# Access a specific column by its name
prob_measure_0_prep1 = df["Prob meas0 prep1 "]

prob_measure_1_prep0 = df["Prob meas1 prep0 "] 

# Convert the column to a list if needed
meas0prep1_list = prob_measure_0_prep1.tolist()
meas1prep0_list = prob_measure_1_prep0.tolist()

print(prob_measure_0_prep1)
print(prob_measure_1_prep0)

t1_values = []
t2_values = []
frequency_values = []
anharmonicity_values = []
readout_assignment_error = []
sx_gate_error = []

qubits = properties.qubits
num_qubits = len(qubits)

# Loop through qubits to extract properties
for qubit in range(num_qubits):
    qubit_properties = properties.qubit_property(qubit)
    t1 = properties.t1(qubit)
    t2 = properties.t2(qubit)
    frequency = properties.frequency(qubit)
    anharmonicity = qubit_properties.get('anharmonicity', None)[0]

    t1_values.append(t1)
    t2_values.append(t2)
    frequency_values.append(frequency)
    anharmonicity_values.append(anharmonicity)

    # Readout assignment error (typically taken from the readout error section)
    readout_error = properties.readout_error(qubit)
    readout_assignment_error.append(readout_error)

    # Gate error (specific to the SX gate)
    sx_error = properties.gate_error('sx', qubit)
    sx_gate_error.append(sx_error)

# Create a DataFrame for better analysis
data = {
    'Qubit': range(num_qubits),
    'T1': t1_values,
    'T2': t2_values,
    'Frequency (GHz)': frequency_values,
    'Anharmonicity (GHz)': anharmonicity_values,
    'Readout Assignment Error': readout_assignment_error,
    'SX Gate Error': sx_gate_error,
    'Prob meas 0 prep1': meas0prep1_list,
    'Prob meas 1 prep0': meas1prep0_list
}

df = pd.DataFrame(data)

df['T1'] = pd.to_numeric(df['T1'], errors='coerce')
df['T2'] = pd.to_numeric(df['T2'], errors='coerce')


# Check again the data types
print(df.dtypes)
print(df)

0      0.0218
1      0.0428
2      0.0102
3      0.0152
4      0.0194
        ...  
122    0.0196
123    0.0128
124    0.0074
125    0.0524
126    0.0090
Name: Prob meas0 prep1 , Length: 127, dtype: float64
0      0.0254
1      0.0478
2      0.0066
3      0.0434
4      0.0124
        ...  
122    0.0212
123    0.0098
124    0.0186
125    0.0092
126    0.0078
Name: Prob meas1 prep0 , Length: 127, dtype: float64
Qubit                         int64
T1                          float64
T2                          float64
Frequency (GHz)             float64
Anharmonicity (GHz)         float64
Readout Assignment Error    float64
SX Gate Error               float64
Prob meas 0 prep1           float64
Prob meas 1 prep0           float64
dtype: object
     Qubit        T1        T2  Frequency (GHz)  Anharmonicity (GHz)  \
0        0  0.000258  0.000059     4.721908e+09        -3.119787e+08   
1        1  0.000402  0.000278     4.815136e+09        -3.097440e+08   
2        2  0.000290  0.000329  

In [ ]:
import matplotlib.pyplot as plt

# Plot T1 and T2 values
plt.figure(figsize=(12, 6))
plt.bar(df['Qubit'], df['T1'], label='T1 (us)', alpha=0.7)
plt.bar(df['Qubit'], df['T2'], label='T2 (us)', alpha=0.5)
plt.xlabel('Qubit')
plt.ylabel('Time (us)')
plt.title('T1 and T2 Values for Each Qubit')
plt.legend()
plt.show()

# Plot Frequency
plt.figure(figsize=(12, 6))
plt.bar(df['Qubit'], df['Frequency (GHz)'], label='Frequency (GHz)', color='orange')
plt.xlabel('Qubit')
plt.ylabel('Frequency (GHz)')
plt.title('Frequency Values for Each Qubit')
plt.show()

# Plot Anharmonicity
plt.figure(figsize=(12, 6))
plt.bar(df['Qubit'], df['Anharmonicity (GHz)'], label='Anharmonicity (GHz)', color='green')
plt.xlabel('Qubit')
plt.ylabel('Anharmonicity (GHz)')
plt.title('Anharmonicity Values for Each Qubit')
plt.show()

In [7]:
columns_of_interest_noise = ['T1', 'T2', 'Frequency (GHz)', 'Anharmonicity (GHz)']
columns_of_interest_fidelity = ['Readout Assignment Error', 'SX Gate Error', 'Prob meas 0 prep1', 'Prob meas 1 prep0']

noise_subset_df = df[columns_of_interest_noise]
fidelity_subset_df = df[columns_of_interest_fidelity]
noise_correlation_matrix = noise_subset_df.corr()
fidelity_correlation_matrix = fidelity_subset_df.corr()

print("Noise Correlation Matrix:\n", noise_correlation_matrix)
print("Fidelity Correlation Matrix:\n", fidelity_correlation_matrix)


Noise Correlation Matrix:
                            T1        T2  Frequency (GHz)  Anharmonicity (GHz)
T1                   1.000000  0.260017        -0.178893            -0.028149
T2                   0.260017  1.000000        -0.118568             0.088635
Frequency (GHz)     -0.178893 -0.118568         1.000000             0.179237
Anharmonicity (GHz) -0.028149  0.088635         0.179237             1.000000
Fidelity Correlation Matrix:
                           Readout Assignment Error  SX Gate Error  \
Readout Assignment Error                  1.000000       0.098390   
SX Gate Error                             0.098390       1.000000   
Prob meas 0 prep1                         0.894565       0.247692   
Prob meas 1 prep0                         0.915343      -0.053802   

                          Prob meas 0 prep1  Prob meas 1 prep0  
Readout Assignment Error           0.894565           0.915343  
SX Gate Error                      0.247692          -0.053802  
Prob meas 0 

In [ ]:
print(properties.toDict())